In [1]:
from pyspark.sql import SparkSession
from operator import add

spark_session = SparkSession.builder\
        .master("spark://192.168.2.250:7077") \
        .appName("samir_mehdiyev_A3")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores", 2)\
        .config("spark.driver.port",9999)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()

# RDD API
spark_context = spark_session.sparkContext

spark_context.setLogLevel("INFO")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/19 16:54:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/02/19 16:54:51 WARN Utils: Service 'sparkDriver' could not bind on port 9999. Attempting port 10000.
24/02/19 16:54:52 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/02/19 16:54:52 WARN Utils: Service 'org.apache.spark.network.netty.NettyBlockTransferService' could not bind on port 10005. Attempting port 10006.


In [2]:
# B.1 Load the CSV file from HDFS, and call show() to verify the data is loaded correctly.
csv_file_path = 'hdfs://192.168.2.250:9000/parking-citations.csv'
df = spark_session.read.option("header",True) \
    .csv(csv_file_path)
df.show()

24/02/19 16:54:53 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
24/02/19 16:54:53 INFO SharedState: Warehouse path is 'file:/home/ubuntu/spark-warehouse'.
24/02/19 16:54:54 INFO InMemoryFileIndex: It took 137 ms to list leaf files for 1 paths.
24/02/19 16:54:54 INFO InMemoryFileIndex: It took 23 ms to list leaf files for 1 paths.
24/02/19 16:54:57 INFO FileSourceStrategy: Pushed Filters: 
24/02/19 16:54:57 INFO FileSourceStrategy: Post-Scan Filters: (length(trim(value#0, None)) > 0)
24/02/19 16:54:58 INFO CodeGenerator: Code generated in 321.692717 ms
24/02/19 16:54:58 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 200.5 KiB, free 434.2 MiB)
24/02/19 16:54:58 INFO MemoryStore: Block broadcast_0_piece0 stored as bytes in memory (estimated size 34.5 KiB, free 434.2 MiB)
24/02/19 16:54:58 INFO BlockManagerInfo: Added broadcast_0_piece0 in memory on host-192-168-2-215-de1:10006 (size: 34.5 KiB, free:

+-------------+--------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+---------+---------+------------------+-----------------+----------------------+
|Ticket number|          Issue Date|Issue time|Meter Id|Marked Time|RP State Plate|Plate Expiry Date| VIN|Make|Body Style|Color|            Location|Route|Agency|Violation code|Violation Description|Fine amount| Latitude|Longitude|Agency Description|Color Description|Body Style Description|
+-------------+--------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+---------+---------+------------------+-----------------+----------------------+
|   1103341116|2015-12-21T00:00:...|      1251|    NULL|       NULL|            CA|           200304|NULL|HOND|        PA|  

24/02/19 16:55:05 INFO CodeGenerator: Code generated in 63.572545 ms


In [3]:
# B.2 Print the schema for the DataFrame.
df.printSchema()

root
 |-- Ticket number: string (nullable = true)
 |-- Issue Date: string (nullable = true)
 |-- Issue time: string (nullable = true)
 |-- Meter Id: string (nullable = true)
 |-- Marked Time: string (nullable = true)
 |-- RP State Plate: string (nullable = true)
 |-- Plate Expiry Date: string (nullable = true)
 |-- VIN: string (nullable = true)
 |-- Make: string (nullable = true)
 |-- Body Style: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Route: string (nullable = true)
 |-- Agency: string (nullable = true)
 |-- Violation code: string (nullable = true)
 |-- Violation Description: string (nullable = true)
 |-- Fine amount: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Agency Description: string (nullable = true)
 |-- Color Description: string (nullable = true)
 |-- Body Style Description: string (nullable = true)



In [4]:
# B.3 Count the number of rows in the CSV file.
row_count = df.count()
print(f"Number of rows: {row_count}")

24/02/19 16:55:05 INFO FileSourceStrategy: Pushed Filters: 
24/02/19 16:55:05 INFO FileSourceStrategy: Post-Scan Filters: 
24/02/19 16:55:05 INFO BlockManagerInfo: Removed broadcast_3_piece0 on 192.168.2.249:10007 in memory (size: 34.5 KiB, free: 434.4 MiB)
24/02/19 16:55:05 INFO CodeGenerator: Code generated in 23.774004 ms
24/02/19 16:55:05 INFO BlockManagerInfo: Removed broadcast_3_piece0 on host-192-168-2-215-de1:10006 in memory (size: 34.5 KiB, free: 434.3 MiB)
24/02/19 16:55:05 INFO MemoryStore: Block broadcast_5 stored as values in memory (estimated size 200.4 KiB, free 433.7 MiB)
24/02/19 16:55:05 INFO MemoryStore: Block broadcast_5_piece0 stored as bytes in memory (estimated size 34.5 KiB, free 433.7 MiB)
24/02/19 16:55:05 INFO BlockManagerInfo: Added broadcast_5_piece0 in memory on host-192-168-2-215-de1:10006 (size: 34.5 KiB, free: 434.3 MiB)
24/02/19 16:55:05 INFO SparkContext: Created broadcast 5 from count at NativeMethodAccessorImpl.java:0
24/02/19 16:55:05 INFO FileSour

Number of rows: 13077724


24/02/19 16:55:10 INFO MapOutputTrackerMasterEndpoint: Asked to send map output locations for shuffle 0 to 192.168.2.249:51422
24/02/19 16:55:11 INFO TaskSetManager: Finished task 0.0 in stage 4.0 (TID 18) in 164 ms on 192.168.2.249 (executor 0) (1/1)
24/02/19 16:55:11 INFO TaskSchedulerImpl: Removed TaskSet 4.0, whose tasks have all completed, from pool 
24/02/19 16:55:11 INFO DAGScheduler: ResultStage 4 (count at NativeMethodAccessorImpl.java:0) finished in 0.190 s
24/02/19 16:55:11 INFO DAGScheduler: Job 3 is finished. Cancelling potential speculative or zombie tasks for this job
24/02/19 16:55:11 INFO TaskSchedulerImpl: Killing all running tasks in stage 4: Stage finished
24/02/19 16:55:11 INFO DAGScheduler: Job 3 finished: count at NativeMethodAccessorImpl.java:0, took 0.201471 s


In [5]:
# B.4 Count the number of partitions in the underlying RDD.
num_partitions = df.rdd.getNumPartitions()
print(f"Number of partitions: {num_partitions}")

24/02/19 16:55:11 INFO FileSourceStrategy: Pushed Filters: 
24/02/19 16:55:11 INFO FileSourceStrategy: Post-Scan Filters: 
24/02/19 16:55:11 INFO MemoryStore: Block broadcast_8 stored as values in memory (estimated size 200.4 KiB, free 433.7 MiB)
24/02/19 16:55:11 INFO MemoryStore: Block broadcast_8_piece0 stored as bytes in memory (estimated size 34.5 KiB, free 433.7 MiB)
24/02/19 16:55:11 INFO BlockManagerInfo: Added broadcast_8_piece0 in memory on host-192-168-2-215-de1:10006 (size: 34.5 KiB, free: 434.3 MiB)
24/02/19 16:55:11 INFO SparkContext: Created broadcast 8 from javaToPython at NativeMethodAccessorImpl.java:0
24/02/19 16:55:11 INFO FileSourceScanExec: Planning scan with bin packing, max size: 134217728 bytes, open cost is considered as scanning 4194304 bytes.


Number of partitions: 16


In [6]:
# B.5 Drop the columns VIN, Latitude and Longitude.
df = df.drop('VIN', 'Latitude', 'Longitude')

In [7]:
# B.6 Find the maximum fine amount. How many fines have this amount?
# You need to convert the ‘fine amount’ column to a float to do this correctly.
from pyspark.sql.functions import col, max

df = df.withColumn("Fine amount", col("Fine amount").cast("float"))
max_fine = df.agg(max("Fine amount")).collect()[0][0]
print(f"Maximum fine amount: {max_fine}")

max_fine_count = df.filter(col("Fine amount") == max_fine).count()
print(f"Fines with maximum amount: {max_fine_count}")


24/02/19 16:55:11 INFO FileSourceStrategy: Pushed Filters: 
24/02/19 16:55:11 INFO FileSourceStrategy: Post-Scan Filters: 
24/02/19 16:55:11 INFO CodeGenerator: Code generated in 27.135439 ms
24/02/19 16:55:11 INFO MemoryStore: Block broadcast_9 stored as values in memory (estimated size 200.4 KiB, free 433.5 MiB)
24/02/19 16:55:11 INFO BlockManagerInfo: Removed broadcast_5_piece0 on 192.168.2.249:10007 in memory (size: 34.5 KiB, free: 434.4 MiB)
24/02/19 16:55:11 INFO MemoryStore: Block broadcast_9_piece0 stored as bytes in memory (estimated size 34.5 KiB, free 433.4 MiB)
24/02/19 16:55:11 INFO BlockManagerInfo: Added broadcast_9_piece0 in memory on host-192-168-2-215-de1:10006 (size: 34.5 KiB, free: 434.3 MiB)
24/02/19 16:55:11 INFO BlockManagerInfo: Removed broadcast_5_piece0 on host-192-168-2-215-de1:10006 in memory (size: 34.5 KiB, free: 434.3 MiB)
24/02/19 16:55:11 INFO SparkContext: Created broadcast 9 from collect at /tmp/ipykernel_28993/2777613339.py:6
24/02/19 16:55:11 INFO F

Maximum fine amount: 1100.0


24/02/19 16:55:17 INFO FileSourceStrategy: Pushed Filters: IsNotNull(Fine amount)
24/02/19 16:55:17 INFO FileSourceStrategy: Post-Scan Filters: isnotnull(Fine amount#33),(cast(Fine amount#33 as float) = 1100.0)
24/02/19 16:55:17 INFO CodeGenerator: Code generated in 28.065548 ms
24/02/19 16:55:17 INFO MemoryStore: Block broadcast_12 stored as values in memory (estimated size 200.4 KiB, free 433.5 MiB)
24/02/19 16:55:17 INFO MemoryStore: Block broadcast_12_piece0 stored as bytes in memory (estimated size 34.5 KiB, free 433.4 MiB)
24/02/19 16:55:17 INFO BlockManagerInfo: Added broadcast_12_piece0 in memory on host-192-168-2-215-de1:10006 (size: 34.5 KiB, free: 434.2 MiB)
24/02/19 16:55:17 INFO SparkContext: Created broadcast 12 from count at NativeMethodAccessorImpl.java:0
24/02/19 16:55:17 INFO FileSourceScanExec: Planning scan with bin packing, max size: 134217728 bytes, open cost is considered as scanning 4194304 bytes.
24/02/19 16:55:17 INFO DAGScheduler: Registering RDD 36 (count at

Fines with maximum amount: 626


24/02/19 16:55:22 INFO MapOutputTrackerMasterEndpoint: Asked to send map output locations for shuffle 2 to 192.168.2.252:46070
24/02/19 16:55:22 INFO TaskSetManager: Finished task 0.0 in stage 10.0 (TID 52) in 146 ms on 192.168.2.252 (executor 1) (1/1)
24/02/19 16:55:22 INFO TaskSchedulerImpl: Removed TaskSet 10.0, whose tasks have all completed, from pool 
24/02/19 16:55:22 INFO DAGScheduler: ResultStage 10 (count at NativeMethodAccessorImpl.java:0) finished in 0.158 s
24/02/19 16:55:22 INFO DAGScheduler: Job 7 is finished. Cancelling potential speculative or zombie tasks for this job
24/02/19 16:55:22 INFO TaskSchedulerImpl: Killing all running tasks in stage 10: Stage finished
24/02/19 16:55:22 INFO DAGScheduler: Job 7 finished: count at NativeMethodAccessorImpl.java:0, took 0.165100 s


In [8]:
# B.7 Show the top 20 most frequent vehicle makes, and their frequencies.
from pyspark.sql.functions import desc

df.groupBy("Make").count().orderBy(desc("count")).show(20)

24/02/19 16:55:23 INFO FileSourceStrategy: Pushed Filters: 
24/02/19 16:55:23 INFO FileSourceStrategy: Post-Scan Filters: 
24/02/19 16:55:23 INFO CodeGenerator: Code generated in 111.684146 ms
24/02/19 16:55:23 INFO MemoryStore: Block broadcast_15 stored as values in memory (estimated size 200.4 KiB, free 433.5 MiB)
24/02/19 16:55:23 INFO MemoryStore: Block broadcast_15_piece0 stored as bytes in memory (estimated size 34.5 KiB, free 433.4 MiB)
24/02/19 16:55:23 INFO BlockManagerInfo: Added broadcast_15_piece0 in memory on host-192-168-2-215-de1:10006 (size: 34.5 KiB, free: 434.2 MiB)
24/02/19 16:55:23 INFO SparkContext: Created broadcast 15 from showString at NativeMethodAccessorImpl.java:0
24/02/19 16:55:23 INFO FileSourceScanExec: Planning scan with bin packing, max size: 134217728 bytes, open cost is considered as scanning 4194304 bytes.
24/02/19 16:55:23 INFO DAGScheduler: Registering RDD 43 (showString at NativeMethodAccessorImpl.java:0) as input to shuffle 3
24/02/19 16:55:23 INF

+----+-------+
|Make|  count|
+----+-------+
|TOYT|2150768|
|HOND|1479996|
|FORD|1116235|
|NISS| 945133|
|CHEV| 892676|
| BMW| 603092|
|MERZ| 543298|
|VOLK| 432030|
|HYUN| 404917|
|DODG| 391686|
|LEXS| 368420|
| KIA| 328155|
|JEEP| 316300|
|AUDI| 255395|
|MAZD| 242344|
|OTHR| 205546|
| GMC| 184889|
|INFI| 174315|
|CHRY| 159948|
|SUBA| 154640|
+----+-------+
only showing top 20 rows



24/02/19 16:55:28 INFO TaskSetManager: Finished task 0.0 in stage 13.0 (TID 69) in 150 ms on 192.168.2.252 (executor 1) (1/1)
24/02/19 16:55:28 INFO TaskSchedulerImpl: Removed TaskSet 13.0, whose tasks have all completed, from pool 
24/02/19 16:55:28 INFO DAGScheduler: ResultStage 13 (showString at NativeMethodAccessorImpl.java:0) finished in 0.163 s
24/02/19 16:55:28 INFO DAGScheduler: Job 9 is finished. Cancelling potential speculative or zombie tasks for this job
24/02/19 16:55:28 INFO TaskSchedulerImpl: Killing all running tasks in stage 13: Stage finished
24/02/19 16:55:28 INFO DAGScheduler: Job 9 finished: showString at NativeMethodAccessorImpl.java:0, took 0.195337 s
24/02/19 16:55:28 INFO CodeGenerator: Code generated in 17.86223 ms             
24/02/19 16:55:28 INFO CodeGenerator: Code generated in 8.016321 ms


In [9]:
# B.8 Let’s expand some abbreviations in the color column. Create a User Defined Function to
# create a new column, ‘color long’, mapping the original colors to their corresponding values
# in the dictionary below. If there is no key matching the original color, use the original color.

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

COLORS = {
    'AL':'Aluminum', 'AM':'Amber', 'BG':'Beige', 'BK':'Black',
    'BL':'Blue', 'BN':'Brown', 'BR':'Brown', 'BZ':'Bronze',
    'CH':'Charcoal', 'DK':'Dark', 'GD':'Gold', 'GO':'Gold',
    'GN':'Green', 'GY':'Gray', 'GT':'Granite', 'IV':'Ivory',
    'LT':'Light', 'OL':'Olive', 'OR':'Orange', 'MR':'Maroon',
    'PK':'Pink', 'RD':'Red', 'RE':'Red', 'SI':'Silver', 'SL':'Silver',
    'SM':'Smoke', 'TN':'Tan', 'VT':'Violet', 'WT':'White', 'WH':'White',
    'YL':'Yellow', 'YE':'Yellow', 'UN':'Unknown'
    }

# Define the UDF
def expand_color_abbr(color_abbr):
    return COLORS.get(color_abbr, color_abbr)

expand_color_udf = udf(expand_color_abbr, StringType())

# Create a new column with expanded colors
df = df.withColumn("color long", expand_color_udf(col("Color")))

In [10]:
# B.9 Using this new column, what’s the most frequent colour value for Toyotas (TOYT)?
df.filter(df.Make == "TOYT").groupBy("color long").count().orderBy(desc("count")).show(1)


24/02/19 16:55:29 INFO FileSourceStrategy: Pushed Filters: IsNotNull(Make),EqualTo(Make,TOYT)
24/02/19 16:55:29 INFO FileSourceStrategy: Post-Scan Filters: isnotnull(Make#25),(Make#25 = TOYT)
24/02/19 16:55:29 INFO CodeGenerator: Code generated in 62.045544 ms
24/02/19 16:55:29 INFO CodeGenerator: Code generated in 16.657631 ms
24/02/19 16:55:29 INFO MemoryStore: Block broadcast_18 stored as values in memory (estimated size 200.4 KiB, free 433.4 MiB)
24/02/19 16:55:29 INFO MemoryStore: Block broadcast_18_piece0 stored as bytes in memory (estimated size 34.5 KiB, free 433.4 MiB)
24/02/19 16:55:29 INFO BlockManagerInfo: Added broadcast_18_piece0 in memory on host-192-168-2-215-de1:10006 (size: 34.5 KiB, free: 434.2 MiB)
24/02/19 16:55:29 INFO SparkContext: Created broadcast 18 from showString at NativeMethodAccessorImpl.java:0
24/02/19 16:55:29 INFO FileSourceScanExec: Planning scan with bin packing, max size: 134217728 bytes, open cost is considered as scanning 4194304 bytes.
24/02/19 1

+----------+------+
|color long| count|
+----------+------+
|      Gray|489697|
+----------+------+
only showing top 1 row



24/02/19 16:55:34 INFO SparkContext: Starting job: showString at NativeMethodAccessorImpl.java:0
24/02/19 16:55:34 INFO DAGScheduler: Got job 11 (showString at NativeMethodAccessorImpl.java:0) with 1 output partitions
24/02/19 16:55:34 INFO DAGScheduler: Final stage: ResultStage 16 (showString at NativeMethodAccessorImpl.java:0)
24/02/19 16:55:34 INFO DAGScheduler: Parents of final stage: List(ShuffleMapStage 15)
24/02/19 16:55:34 INFO DAGScheduler: Missing parents: List()
24/02/19 16:55:34 INFO DAGScheduler: Submitting ResultStage 16 (MapPartitionsRDD[58] at showString at NativeMethodAccessorImpl.java:0), which has no missing parents
24/02/19 16:55:34 INFO MemoryStore: Block broadcast_20 stored as values in memory (estimated size 48.1 KiB, free 433.6 MiB)
24/02/19 16:55:34 INFO MemoryStore: Block broadcast_20_piece0 stored as bytes in memory (estimated size 23.2 KiB, free 433.6 MiB)
24/02/19 16:55:34 INFO BlockManagerInfo: Added broadcast_20_piece0 in memory on host-192-168-2-215-de1:

In [11]:
spark_context.stop()

24/02/19 16:55:34 INFO SparkContext: SparkContext is stopping with exitCode 0.
24/02/19 16:55:34 INFO SparkUI: Stopped Spark web UI at http://host-192-168-2-215-de1:4041
24/02/19 16:55:34 INFO StandaloneSchedulerBackend: Shutting down all executors
24/02/19 16:55:34 INFO StandaloneSchedulerBackend$StandaloneDriverEndpoint: Asking each executor to shut down
24/02/19 16:55:34 INFO MapOutputTrackerMasterEndpoint: MapOutputTrackerMasterEndpoint stopped!
24/02/19 16:55:34 INFO MemoryStore: MemoryStore cleared
24/02/19 16:55:34 INFO BlockManager: BlockManager stopped
24/02/19 16:55:34 INFO BlockManagerMaster: BlockManagerMaster stopped
24/02/19 16:55:34 INFO OutputCommitCoordinator$OutputCommitCoordinatorEndpoint: OutputCommitCoordinator stopped!
24/02/19 16:55:34 INFO SparkContext: Successfully stopped SparkContext
